<a href="https://colab.research.google.com/github/myazann/Text-Generation/blob/main/Conversation_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets==1.0.2

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
! pip install tokenizers

import os
import csv
import json
import pandas as pd
import numpy as np
from transformers import OpenAIGPTLMHeadModel, Trainer, TrainingArguments, GPT2LMHeadModel
from transformers import AutoModel, AutoModelWithLMHead, AutoTokenizer, EncoderDecoderModel, BertTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling,LineByLineTextDataset
from sklearn.model_selection import train_test_split
from transformers import pipeline
import torch
import gc
import datasets
from dataclasses import dataclass, field
from typing import Optional
from tokenizers import BertWordPieceTokenizer, Tokenizer
from tokenizers.processors import BertProcessing

Getting the data into a dataframe.



In [ ]:
os.chdir("/content/drive/My Drive")

!cp ChatExport_2021-01-01.zip /content

os.chdir("/content")

!unzip ChatExport_2021-01-01.zip
!rm ChatExport_2021-01-01.zip

In [ ]:
f = open('ChatExport_2021-01-01/result.json',encoding="utf8") 
messages = json.load(f)

orig_df = pd.DataFrame(columns = ["Writer","Messages","Date"])
i = 0

for message in messages["messages"]:
    if message["type"] == "message":
        orig_df = orig_df.append({
            "Writer":message["from"],
            "Messages":message["text"],
            "Date":message["date"]
                  }, ignore_index = True)
        if i%100 == 0:
            print(i)
        i += 1


Preprocessing

In [ ]:
data_df = orig_df

data_df["Writer"] = np.where(data_df["Writer"].isna(), "Aykut", data_df["Writer"])

data_df["Messages_Lower"] = data_df["Messages"].str.lower()
data_df["Messages_Lower"] = data_df["Messages_Lower"].str.strip()
data_df["Word_Splits"] = data_df["Messages_Lower"].str.split()

data_df = data_df.dropna()

data_df["Messages"] = data_df["Messages_Lower"]

data_df = data_df.drop("Messages_Lower", axis = 1)

data_df = data_df.loc[data_df["Messages"] != ""]

writers = ["Mert Yazan", "Kumru", "Atamert", "Abidin Dekar", "Aykut"]

data_df = data_df.loc[data_df["Writer"].isin(writers)]

i = 0
data_lengths = []
msg_words = data_df["Word_Splits"].values

while i < len(msg_words):
    data_lengths.append(len(msg_words[i]))        
    i += 1 
    
data_df["Lengths"] = data_lengths    

data_df = data_df.loc[data_df["Lengths"] >= 1]

data_df["Messages"] = np.where((data_df["Messages"].str.contains("hahaha")) & (data_df["Lengths"] == 1),"hahaha",data_df["Messages"])

data_df["Writer"] = np.where(data_df["Writer"] == "Mert Yazan", "Mert", data_df["Writer"])
data_df["Writer"] = np.where(data_df["Writer"] == "Abidin Dekar", "Abidin", data_df["Writer"])

data_df["input"] = data_df["Writer"] + " " + data_df["Messages"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Creating the labels as the next sentences, and splitting them into train and val sets.

In [ ]:
label_len = len(data_df["input"].values)-1

data = []
labels = []
text = ""

for i in range(label_len):
  labels.append(data_df["input"].values[i+1])
  for j in range (1):
    text = text + " " + data_df["input"].values[i+j] 
  data.append(text)
  text = ""

labels = np.asarray(labels)
data = np.asarray(data)

df2 = pd.DataFrame()

df2["Written_Text"] = data
df2["Answer_Text"] = labels

train, val = train_test_split(df2, test_size = 0.2)

train.to_csv("telegram_chatbot_train.csv", index = False, encoding = "utf-8")

val.to_csv("telegram_chatbot_val.csv", index = False, encoding = "utf-8")

Train a tokenizer from scratch.

In [ ]:
tokenizer = BertWordPieceTokenizer()

tokenizer.train(files="/content/drive/My Drive/telegram_chatbot_train.csv", vocab_size=32_000, min_frequency=2)
tokenizer.save_model(".")

['./vocab.txt']

Loading data, tokenizer and the model.

In [2]:
tg_data_train = datasets.load_dataset("csv", data_files = "/content/drive/My Drive/telegram_chatbot_train.csv", split = "train")
tg_data_val = datasets.load_dataset("csv", data_files = "/content/drive/My Drive/telegram_chatbot_val.csv", split = "train")

tokenizer = BertTokenizer.from_pretrained("/content/drive/My Drive/vocab.txt", do_lower_case=True, 
                                          return_special_tokens_mask=True, model_max_len = 512, is_fast = True)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

##tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased', do_lower_case=True, return_special_tokens_mask=True)
##tokenizer.bos_token = tokenizer.cls_token
##tokenizer.eos_token = tokenizer.sep_token

model = EncoderDecoderModel.from_encoder_decoder_pretrained("dbmdz/bert-base-turkish-cased", "dbmdz/bert-base-turkish-cased")
model = model.from_pretrained("/content/drive/My Drive/enc2dec/checkpoint-100000")

Using custom data configuration default


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-52f8e7c0f5270178/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277. Subsequent calls will reuse this data.


Using custom data configuration default


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-36c2692e7da1159c/0.0.0/0d06ce3712951dae7909fb214283b88efab3578535edb5eebd37c498b7a35277. Subsequent calls will reuse this data.


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bi

Free memory if necessary.

In [ ]:
model = None
trainer = None
training_args = None

del model
del training_args
del trainer

gc.collect()

with torch.no_grad():
    torch.cuda.empty_cache()

Creating the data pipeline.

In [3]:
def process_data_to_model_inputs(batch):
  inputs = tokenizer(batch["Written_Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["Answer_Text"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [4]:
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = 150
model.config.min_length = 5

model.config.no_repeat_ngram_size = 3 
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4


batch_size=16
encoder_max_length=128
decoder_max_length=128


tg_data_train = tg_data_train.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size,
    remove_columns=["Written_Text", "Answer_Text"]
    )

tg_data_train.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

tg_data_val = tg_data_val.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Written_Text", "Answer_Text"]
)
tg_data_val.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Instantiate a Trainer and start training.

In [5]:
training_args = TrainingArguments(
    output_dir="./enc2dec",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=64,
    ##predict_with_generate=True,
    ##evaluation_strategy = "steps",
    do_train=True,
    do_eval=True,
    
    logging_steps=2500,  
    save_steps=2500,  
    eval_steps=2500,  
    warmup_steps=2500,  
    max_steps=200000, 
    overwrite_output_dir=True,
    fp16=True
)

# instantiate trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tg_data_train,
    eval_dataset=tg_data_val,
)

In [ ]:
trainer.train("/content/drive/My Drive/enc2dec/checkpoint-100000")

/usr/local/lib/python3.7/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Step,Training Loss
102500,2.177400
105000,2.041700


In [ ]:
trainer.evaluate()

Sample code to generate text given an input. Different decoding strategies can be used.


In [ ]:
start = tokenizer("aykut mert", return_tensors="pt").input_ids
out = model.generate(start.cuda(), min_length = 5, max_length = 150, 
                     
                    ## top k-sampling
                    ##top_k = 50, do_sample = True  
                     
                    ## top p-sampling
                    ##top_p = 0.95, do_sample = True 

                    #beam search
                    no_repeat_ngram_size = 3, early_stopping=True, num_beams = 4, length_penalty = 2.0

                    ## temperature=0.75,
                    ##num_return_sequences = 3
                     )

decoded = tokenizer.batch_decode(out)[0].split()
kisi = decoded[1]
msg = decoded[2:-1]

kisi + ":" + " ".join(([str(elem) for elem in msg]))

Start a conversation. 

In [ ]:
i = 0
start_sentence = input()

while i < 25:

  start = tokenizer(start_sentence, return_tensors="pt").input_ids
  out = model.generate(start.cuda(),  min_length = 5, max_length = 150,
                     top_p = 0.95, do_sample = True )
  

  decoded = tokenizer.batch_decode(out)[0]

  kisi = decoded.split()[1]
  msg = decoded.split()[2:-1]

  print(kisi + ":" + " ".join(([str(elem) for elem in msg])))
  start_sentence = decoded

  i += 1

In [ ]:
!rm -rf runs